For this project, I first tried using AI, and then switched to modifying the code of another person. The original code can be found at the following URL: https://www.kaggle.com/code/vijayaadithyanvg/car-price-prediction-used-cars

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn import metrics

In [32]:
car_df = pd.read_csv('cars.csv')
# Extract brand from name column
car_df.insert(0, "make", car_df['name'].apply(lambda x: x.split()[0]))
car_df.insert(1, "model", car_df['name'].apply(lambda x: " ".join(x.split()[1:])))
car_df = car_df.drop(columns='name', axis=1)
car_df['selling_price'] = car_df['selling_price'].multiply(0.012)
make = car_df['make'].unique()
make_dict = {}
for i in range(len(make)):
    make_dict[make[i]] = i
car_df['make'] = car_df['make'].map(make_dict)
car_df.sort_values(by='selling_price', ascending=False)
print(make_dict)
car_df

{'Maruti': 0, 'Skoda': 1, 'Honda': 2, 'Hyundai': 3, 'Toyota': 4, 'Ford': 5, 'Renault': 6, 'Mahindra': 7, 'Tata': 8, 'Chevrolet': 9, 'Fiat': 10, 'Datsun': 11, 'Jeep': 12, 'Mercedes-Benz': 13, 'Mitsubishi': 14, 'Audi': 15, 'Volkswagen': 16, 'BMW': 17, 'Nissan': 18, 'Lexus': 19, 'Jaguar': 20, 'Land': 21, 'MG': 22, 'Volvo': 23, 'Daewoo': 24, 'Kia': 25, 'Force': 26, 'Ambassador': 27, 'Ashok': 28, 'Isuzu': 29, 'Opel': 30, 'Peugeot': 31}


,make,model,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,0,Swift Dzire VDI,2014,5400.0,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,1,Rapid 1.5 TDI Ambition,2014,4440.0,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,2,City 2017-2020 EXi,2006,1896.0,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,3,i20 Sportz Diesel,2010,2700.0,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,0,Swift VXI BSIII,2007,1560.0,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,3,i20 Magna,2013,3840.0,110000,Petrol,Individual,Manual,First Owner,18.5 kmpl,1197 CC,82.85 bhp,113.7Nm@ 4000rpm,5.0
8124,3,Verna CRDi SX,2007,1620.0,119000,Diesel,Individual,Manual,Fourth & Above Owner,16.8 kmpl,1493 CC,110 bhp,"24@ 1,900-2,750(kgm@ rpm)",5.0
8125,0,Swift Dzire ZDi,2009,4584.0,120000,Diesel,Individual,Manual,First Owner,19.3 kmpl,1248 CC,73.9 bhp,190Nm@ 2000rpm,5.0
8126,8,Indigo CR4,2013,3480.0,25000,Diesel,Individual,Manual,First Owner,23.57 kmpl,1396 CC,70 bhp,140Nm@ 1800-3000rpm,5.0


In [25]:
car_df.shape
#rows, then columns

(8128, 14)

In [26]:
car_df.isnull().sum()

make               0
model              0
year               0
selling_price      0
km_driven          0
fuel               0
seller_type        0
transmission       0
owner              0
mileage          221
engine           221
max_power        215
torque           222
seats            221
dtype: int64

For this analysis, I don't really care about the columns with missing data, so I don't need to drop those records.

In [27]:
car_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   make           8128 non-null   object 
 1   model          8128 non-null   object 
 2   year           8128 non-null   int64  
 3   selling_price  8128 non-null   float64
 4   km_driven      8128 non-null   int64  
 5   fuel           8128 non-null   object 
 6   seller_type    8128 non-null   object 
 7   transmission   8128 non-null   object 
 8   owner          8128 non-null   object 
 9   mileage        7907 non-null   object 
 10  engine         7907 non-null   object 
 11  max_power      7913 non-null   object 
 12  torque         7906 non-null   object 
 13  seats          7907 non-null   float64
dtypes: float64(2), int64(2), object(10)
memory usage: 889.1+ KB


In [28]:
print(car_df.selling_price.value_counts())

selling_price
3600.000    227
7200.000    213
4200.000    209
6600.000    204
5400.000    195
           ... 
2435.988      1
6492.000      1
3048.000      1
6336.000      1
1224.000      1
Name: count, Length: 677, dtype: int64
